## Transpo got problems

There are three factories and their existing supply of a good. 

$ S = \begin{bmatrix}
2800\\
5200\\
5800
\end{bmatrix} $

There are seven retailers and their demand for the same good.

$ R = \begin{bmatrix}
1800\\
2400\\
1200\\
800\\
3400\\
2200\\
2000
\end{bmatrix} $

The matrix $C$ below summarizes the transportaiton cost (per unit) of the good from each factory to ech retailer.
 
$ C = \begin{bmatrix}
78 & 28 & 22 & 28 & 32 & 164 & 16\\
54 & 0 & 24 & 0 & 52 & 190 & 34\\
48 & 28 & 34 & 26 & 56 & 198 & 40
\end{bmatrix} $

Let $X$ be a $ 3 \times 7 $ matrix where $x_{ij}$ is the quantity transported from Supplier $i$ to Retailer $j$. 

Constraints:

Two routes are not available and shown missing transportaiton costs. They are marked as 0 in the matrix.

$x_{B2} = 0, x_{B4} = 0 $

The objective is to distribute the good from the suppliers to the retailers at the minimal possible cost. 

$ \min \sum \sum x_{s,r} C_{s,r} $


The demand of each retailer must be met.


In [11]:
using JuMP, GLPK

# Supplier inventory
S = [
    2800
    5200
    5800
]

# Retailer demand
R = [
    1800
    2400
    1200
    800
    3400
    2200
    2000
]

# Cost of transportation
C = [
    78  28  22  28  32  164 16;
    54  0   24  0   52  190 34;
    48  28  34  26  56  198 40;
]

n_retailers = length(R)
n_suppliers = length(S)

# Specify model
m = Model(GLPK.Optimizer)

# Define variables
@variable(m, x[1:n_suppliers,1:n_retailers] >= 0)

# Define objective
@objective(m, Min,
    sum(x[s,r]*C[s,r]
        for s=1:n_suppliers,
            r=1:n_retailers))

# Define constraint: two routes are not available
@constraint(m, x[2,2] == 0)
@constraint(m, x[2,4] == 0)

# Define constraint: Retailer demand is met
for r=1:n_retailers
    @constraint(m, sum(x[s,r] for s=1:n_suppliers) == R[r])
end

# Define constraint: Suppliers aren't overdrawn
for s=1:n_suppliers
    @constraint(m, sum(x[s,r] for r=1:n_retailers) == S[s])
end

# Print model
print(m)

# Set forth and optimize
@time begin
    status = optimize!(m)
end

Min 78 x[1,1] + 28 x[1,2] + 22 x[1,3] + 28 x[1,4] + 32 x[1,5] + 164 x[1,6] + 16 x[1,7] + 54 x[2,1] + 24 x[2,3] + 52 x[2,5] + 190 x[2,6] + 34 x[2,7] + 48 x[3,1] + 28 x[3,2] + 34 x[3,3] + 26 x[3,4] + 56 x[3,5] + 198 x[3,6] + 40 x[3,7]
Subject to
 x[2,2] = 0.0
 x[2,4] = 0.0
 x[1,1] + x[2,1] + x[3,1] = 1800.0
 x[1,2] + x[2,2] + x[3,2] = 2400.0
 x[1,3] + x[2,3] + x[3,3] = 1200.0
 x[1,4] + x[2,4] + x[3,4] = 800.0
 x[1,5] + x[2,5] + x[3,5] = 3400.0
 x[1,6] + x[2,6] + x[3,6] = 2200.0
 x[1,7] + x[2,7] + x[3,7] = 2000.0
 x[1,1] + x[1,2] + x[1,3] + x[1,4] + x[1,5] + x[1,6] + x[1,7] = 2800.0
 x[2,1] + x[2,2] + x[2,3] + x[2,4] + x[2,5] + x[2,6] + x[2,7] = 5200.0
 x[3,1] + x[3,2] + x[3,3] + x[3,4] + x[3,5] + x[3,6] + x[3,7] = 5800.0
 x[1,1] ≥ 0.0
 x[2,1] ≥ 0.0
 x[3,1] ≥ 0.0
 x[1,2] ≥ 0.0
 x[2,2] ≥ 0.0
 x[3,2] ≥ 0.0
 x[1,3] ≥ 0.0
 x[2,3] ≥ 0.0
 x[3,3] ≥ 0.0
 x[1,4] ≥ 0.0
 x[2,4] ≥ 0.0
 x[3,4] ≥ 0.0
 x[1,5] ≥ 0.0
 x[2,5] ≥ 0.0
 x[3,5] ≥ 0.0
 x[1,6] ≥ 0.0
 x[2,6] ≥ 0.0
 x[3,6] ≥ 0.0
 x[1,7] ≥ 0.0
 x[2,

In [13]:
obj = JuMP.objective_value(m)
x_star = JuMP.value.(x)

println("Objective value: ", obj)

Objective value: 800000.0


The minimal cost is $800,000.

In [12]:
obj = JuMP.objective_value(m)
x_star = JuMP.value.(x)

println("Objective value: ", obj)

supplier_names = ["A", "B", "C"]
for s=1:n_suppliers, r=1:n_retailers
    s_name = supplier_names[s]
    println("Supplier $s_name => Retailer $r: ", x_star[s, r])
end

Objective value: 800000.0
Supplier A => Retailer 1: 0.0
Supplier A => Retailer 2: 0.0
Supplier A => Retailer 3: 0.0
Supplier A => Retailer 4: 0.0
Supplier A => Retailer 5: 600.0
Supplier A => Retailer 6: 2200.0
Supplier A => Retailer 7: 0.0
Supplier B => Retailer 1: 0.0
Supplier B => Retailer 2: 0.0
Supplier B => Retailer 3: 1200.0
Supplier B => Retailer 4: 0.0
Supplier B => Retailer 5: 2000.0
Supplier B => Retailer 6: 0.0
Supplier B => Retailer 7: 2000.0
Supplier C => Retailer 1: 1800.0
Supplier C => Retailer 2: 2400.0
Supplier C => Retailer 3: 0.0
Supplier C => Retailer 4: 800.0
Supplier C => Retailer 5: 800.0
Supplier C => Retailer 6: 0.0
Supplier C => Retailer 7: 0.0


Below is a table showing the number of goods distributed from each supplier to each retailer. The supplier is indexed by row and the retailer is indexed by column.

In [19]:
x_star

3×7 Matrix{Float64}:
    0.0     0.0     0.0    0.0   600.0  2200.0     0.0
    0.0     0.0  1200.0    0.0  2000.0     0.0  2000.0
 1800.0  2400.0     0.0  800.0   800.0     0.0     0.0